# Feels Protocol Baseline Analysis

This notebook provides exploratory analysis of the Feels protocol using current default parameters.
It demonstrates the core simulation capabilities and provides baseline metrics for comparison
with parameter optimization scenarios.

## Objectives

1. Run baseline simulation with current protocol defaults (98.5% buffer, 1% treasury, 0.5% creator)
2. Analyze floor price dynamics and POMM deployment patterns
3. Examine fee distribution and accumulation patterns
4. Visualize key metrics and establish baseline performance
5. Compare different fee scenarios to understand parameter sensitivity

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime

# Add project root to path
project_root = Path().cwd().parent
sys.path.append(str(project_root))

from feels_sim.config import SimulationConfig
from feels_sim.core import FeelsSimulation
from feels_sim.metrics import (
    analyze_results, 
    create_summary_plots, 
    create_detailed_analysis_plots,
    generate_summary_report,
    calculate_floor_floor_ratio_stats,
    calculate_pomm_efficiency_metrics
)

print("✓ Libraries imported successfully")

## 1. Baseline Simulation with Current Defaults

In [ ]:
# Run baseline simulation with current protocol defaults
print("Running baseline simulation (168 hours)...")

config = SimulationConfig()  # Uses current defaults: 98.5% buffer, 1% treasury, 0.5% creator
print(f"Fee split: {config.buffer_share_pct:.1f}% buffer, {config.treasury_share_pct:.1f}% treasury, {config.creator_share_pct:.1f}% creator")

sim = FeelsSimulation(config)
baseline_results = sim.run(hours=168)  # 1 week

print(f"✓ Simulation completed: {len(baseline_results.snapshots)} snapshots, {len(baseline_results.hourly_aggregates)} hourly aggregates")

In [ ]:
# Analyze baseline results
baseline_analysis = analyze_results(baseline_results)

print("=== BASELINE SIMULATION RESULTS ===")
print(f"Simulation Duration: {baseline_analysis['simulation_hours']:.1f} hours")
print(f"Initial SOL Price: ${baseline_analysis['initial_sol_price']:.2f}")
print(f"Final SOL Price: ${baseline_analysis['final_sol_price']:.2f}")
print(f"Initial Floor Price: ${baseline_analysis['initial_floor_price']:.6f}")
print(f"Final Floor Price: ${baseline_analysis['final_floor_price']:.6f}")
print(f"")
print(f"Floor Growth Rate (Annual): {baseline_analysis['floor_growth_rate_annual']:.2%}")
print(f"Average Floor/Market Ratio: {baseline_analysis['avg_floor_to_market_ratio']:.2%}")
print(f"Total POMM Deployments: {baseline_analysis['pomm_deployments']}")
print(f"Total Trading Volume: {baseline_analysis['total_volume']:,.0f} FeelsSOL")
print(f"Total Fees Collected: {baseline_analysis['total_fees']:,.2f} FeelsSOL")
print(f"Protocol Efficiency: {baseline_analysis['protocol_efficiency']:.6f} USD/FeelsSOL")
print(f"")
print(f"Final Treasury Balance: {baseline_analysis['final_treasury_balance']:,.2f} FeelsSOL")
print(f"Final Buffer Balance: {baseline_analysis['final_buffer_balance']:,.2f} FeelsSOL")
print(f"Final Deployed FeelsSOL: {baseline_analysis['final_deployed_feelssol']:,.2f} FeelsSOL")
print(f"Buffer Routed (Cumulative): {baseline_analysis['buffer_routed_cumulative']:,.2f} FeelsSOL")
print(f"Mint Cumulative: {baseline_analysis['mint_cumulative']:,.2f} FeelsSOL")

## 2. Visualizations and Pattern Analysis

In [ ]:
# Create comprehensive visualizations
create_summary_plots(baseline_results)
plt.suptitle('Baseline Simulation - Current Protocol Defaults (98.5/1/0.5)', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Create detailed hourly analysis
create_detailed_analysis_plots(baseline_results)
plt.suptitle('Baseline Simulation - Hourly Analysis', fontsize=16)
plt.tight_layout()
plt.show()

## 3. Statistical Analysis of Key Metrics

In [ ]:
# Floor/market ratio statistics
ratio_stats = calculate_floor_floor_ratio_stats(baseline_results)

print("=== FLOOR/MARKET RATIO STATISTICS ===")
for stat, value in ratio_stats.items():
    print(f"{stat}: {value:.4f}")

In [ ]:
# POMM efficiency metrics
pomm_metrics = calculate_pomm_efficiency_metrics(baseline_results)

print("=== POMM DEPLOYMENT EFFICIENCY ===")
for metric, value in pomm_metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value}")

## 4. Fee Distribution Analysis

In [ ]:
# Analyze fee distribution patterns
total_fees = baseline_analysis['total_fees']
treasury_fees = baseline_analysis['final_treasury_balance']
buffer_fees = baseline_analysis['buffer_routed_cumulative']

# Calculate creator fees (would need to track separately)
creator_fees = total_fees - treasury_fees - buffer_fees

print("=== FEE DISTRIBUTION ANALYSIS ===")
print(f"Total Fees Collected: {total_fees:,.2f} FeelsSOL")
print(f"")
print(f"Buffer Allocation: {buffer_fees:,.2f} FeelsSOL ({buffer_fees/total_fees*100:.1f}%)")
print(f"Treasury Allocation: {treasury_fees:,.2f} FeelsSOL ({treasury_fees/total_fees*100:.1f}%)")
print(f"Creator Allocation: {creator_fees:,.2f} FeelsSOL ({creator_fees/total_fees*100:.1f}%)")
print(f"")
print(f"Expected vs Actual Distribution:")
print(f"Buffer: Expected {config.buffer_share_pct:.1f}%, Actual {buffer_fees/total_fees*100:.1f}%")
print(f"Treasury: Expected {config.treasury_share_pct:.1f}%, Actual {treasury_fees/total_fees*100:.1f}%")
print(f"Creator: Expected {config.creator_share_pct:.1f}%, Actual {creator_fees/total_fees*100:.1f}%")

## 5. Scenario Comparison Analysis

In [ ]:
# Compare different fee scenarios
scenarios = [
    ("current_default", "Current Default (98.5/1/0.5)"),
    ("protocol_sustainable", "Protocol Sustainable (90/7/3)"),
    ("creator_incentive", "Creator Incentive (90/5/5)"),
    ("balanced_growth", "Balanced Growth (92/5/3)")
]

scenario_results = {}

print("Running scenario comparison analysis...")

for scenario_key, scenario_name in scenarios:
    print(f"Running {scenario_name}...")
    
    config = SimulationConfig.create_fee_scenario(scenario_key)
    sim = FeelsSimulation(config)
    results = sim.run(hours=168)
    analysis = analyze_results(results)
    
    scenario_results[scenario_key] = {
        "name": scenario_name,
        "config": {
            "buffer": config.buffer_share_pct,
            "treasury": config.treasury_share_pct,
            "creator": config.creator_share_pct
        },
        "analysis": analysis
    }
    
    print(f"  Floor growth: {analysis['floor_growth_rate_annual']:.2%}, "
          f"POMM deployments: {analysis['pomm_deployments']}, "
          f"Treasury: {analysis['final_treasury_balance']:.1f}")

print("✓ Scenario comparison completed")

In [ ]:
# Create comparative visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Fee Scenario Comparison', fontsize=16)

scenarios_list = list(scenario_results.keys())
scenario_names = [scenario_results[s]["name"] for s in scenarios_list]

# Floor growth rates
floor_growth = [scenario_results[s]["analysis"]["floor_growth_rate_annual"] for s in scenarios_list]
axes[0, 0].bar(range(len(scenarios_list)), [x*100 for x in floor_growth])
axes[0, 0].set_title('Floor Growth Rate (Annual %)')
axes[0, 0].set_xticks(range(len(scenarios_list)))
axes[0, 0].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Treasury balances
treasury_balances = [scenario_results[s]["analysis"]["final_treasury_balance"] for s in scenarios_list]
axes[0, 1].bar(range(len(scenarios_list)), treasury_balances)
axes[0, 1].set_title('Final Treasury Balance (FeelsSOL)')
axes[0, 1].set_xticks(range(len(scenarios_list)))
axes[0, 1].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# POMM deployments
pomm_deployments = [scenario_results[s]["analysis"]["pomm_deployments"] for s in scenarios_list]
axes[1, 0].bar(range(len(scenarios_list)), pomm_deployments)
axes[1, 0].set_title('Total POMM Deployments')
axes[1, 0].set_xticks(range(len(scenarios_list)))
axes[1, 0].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Protocol efficiency
efficiency = [scenario_results[s]["analysis"]["protocol_efficiency"] for s in scenarios_list]
axes[1, 1].bar(range(len(scenarios_list)), efficiency)
axes[1, 1].set_title('Protocol Efficiency (USD/FeelsSOL)')
axes[1, 1].set_xticks(range(len(scenarios_list)))
axes[1, 1].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Key Insights and Recommendations

In [ ]:
# Generate key insights
print("=== KEY INSIGHTS FROM BASELINE ANALYSIS ===")
print()

# Best performing scenario analysis
best_floor_growth = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["floor_growth_rate_annual"])
best_treasury = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["final_treasury_balance"])
best_efficiency = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["protocol_efficiency"])

print(f"1. Floor Growth Performance:")
print(f"   Best: {best_floor_growth[1]['name']} ({best_floor_growth[1]['analysis']['floor_growth_rate_annual']:.2%} annual)")
print(f"   Current default achieves {baseline_analysis['floor_growth_rate_annual']:.2%} annual growth")
print()

print(f"2. Treasury Accumulation:")
print(f"   Best: {best_treasury[1]['name']} ({best_treasury[1]['analysis']['final_treasury_balance']:.1f} FeelsSOL)")
print(f"   Current default accumulates {baseline_analysis['final_treasury_balance']:.1f} FeelsSOL")
print()

print(f"3. Protocol Efficiency:")
print(f"   Best: {best_efficiency[1]['name']} ({best_efficiency[1]['analysis']['protocol_efficiency']:.6f} USD/FeelsSOL)")
print(f"   Current default achieves {baseline_analysis['protocol_efficiency']:.6f} USD/FeelsSOL efficiency")
print()

print(f"4. Trade-offs Analysis:")
current_config = scenario_results['current_default']['config']
sustainable_config = scenario_results['protocol_sustainable']['config']

treasury_diff = best_treasury[1]['analysis']['final_treasury_balance'] - baseline_analysis['final_treasury_balance']
floor_diff = best_floor_growth[1]['analysis']['floor_growth_rate_annual'] - baseline_analysis['floor_growth_rate_annual']

print(f"   Moving from current default to protocol sustainable:")
print(f"   - Treasury gains: +{treasury_diff:.1f} FeelsSOL")
print(f"   - Floor growth change: {floor_diff:+.2%}")
print(f"   - Buffer allocation change: {sustainable_config['buffer'] - current_config['buffer']:+.1f}%")
print()

print(f"5. POMM Deployment Patterns:")
print(f"   Baseline deployments: {baseline_analysis['pomm_deployments']} over {baseline_analysis['simulation_hours']:.0f} hours")
print(f"   Deployment frequency: {baseline_analysis['pomm_deployments'] / baseline_analysis['simulation_hours'] * 24:.1f} per day")
print(f"   Average deployment drives floor up by: {(baseline_analysis['final_floor_price'] - baseline_analysis['initial_floor_price']) / baseline_analysis['pomm_deployments']:.8f} USD")

## 7. Export Results for Further Analysis

In [ ]:
# Export results for use in parameter sweep notebook
baseline_export = {
    "timestamp": datetime.now().isoformat(),
    "baseline_config": {
        "treasury_share_pct": config.treasury_share_pct,
        "creator_share_pct": config.creator_share_pct,
        "buffer_share_pct": config.buffer_share_pct,
        "base_fee_bps": config.base_fee_bps,
    },
    "baseline_analysis": baseline_analysis,
    "scenario_comparison": scenario_results,
    "ratio_statistics": ratio_stats,
    "pomm_metrics": pomm_metrics
}

# Save to experiments directory
export_path = project_root / "experiments" / "runs" / "baseline_analysis.json"
export_path.parent.mkdir(parents=True, exist_ok=True)

with open(export_path, 'w') as f:
    json.dump(baseline_export, f, indent=2)

print(f"✓ Baseline analysis results exported to {export_path}")
print(f"✓ Use this data in parameter sweep analysis (02_parameter_sweep.ipynb)")

## Summary

This baseline analysis provides:

1. **Reference Performance Metrics** - Current protocol defaults performance baseline
2. **Fee Distribution Validation** - Confirms actual vs expected fee routing
3. **POMM Deployment Patterns** - Understanding of floor advancement dynamics
4. **Scenario Comparison** - Initial insights into parameter sensitivity
5. **Trade-off Analysis** - Foundation for parameter optimization decisions

**Next Steps:**
- Use `just sweep` to run comprehensive parameter sweeps
- Analyze results in `02_parameter_sweep.ipynb`
- Generate governance recommendations based on findings

**Key Finding:** The current defaults (98.5/1/0.5) prioritize floor advancement over protocol treasury accumulation. Alternative scenarios show the trade-offs available for optimization.